# HOLISTIC

In [71]:
using CSV, Tables, LinearAlgebra, Random, Gurobi, JuMP, Statistics, DataFrames, Metrics
#read train and validation data and split it into X and y
X_train_solar=Matrix(DataFrame(CSV.File("X_train_solar.csv")))
y_train_solar=Matrix(DataFrame(CSV.File("y_train_solar.csv")))
X_train_wind=Matrix(DataFrame(CSV.File("X_train_wind.csv")))
y_train_wind=Matrix(DataFrame(CSV.File("y_train_wind.csv")))

X_valid_solar=Matrix(DataFrame(CSV.File("X_valid_solar.csv")))
y_valid_solar=Matrix(DataFrame(CSV.File("y_valid_solar.csv")))
X_valid_wind=Matrix(DataFrame(CSV.File("X_valid_wind.csv")))
y_valid_wind=Matrix(DataFrame(CSV.File("y_valid_wind.csv")));

### NORMALIZE DATA
We normalize the data with min max scaling. We apply normalization for the numerical features only and then we use alpha to balance between numerical and binary variables.

In [72]:
function min_max_scaling(X, num_feature_indices)
    for i in num_feature_indices
        X[:, i] = (X[:, i] .- minimum(X[:, i])) ./ (maximum(X[:, i]) .- minimum(X[:, i]))
    end
    return X
end

min_max_scaling (generic function with 1 method)

#### SOLAR

In [73]:
#scale SOLAR TRAIN only numerical
X_num_solar=min_max_scaling(Matrix(X_train_solar), 1:15)

# Binary features as they are
X_bin_solar = X_train_solar[:,16:end]

# Numerical vs Categorical features scaling
alpha = 0.75
X_num_solar = alpha*X_num_solar
X_bin_solar = (1-alpha)*X_bin_solar;

# Append data
X_train_solar_norm = [X_num_solar X_bin_solar];

In [74]:
#scale SOLAR VALID only numerical
X_num_solar=min_max_scaling(Matrix(X_valid_solar), 1:15)

# Binary features as they are
X_bin_solar = X_valid_solar[:,16:end]

# Numerical vs Categorical features scaling
alpha = 0.75
X_num_solar = alpha*X_num_solar
X_bin_solar = (1-alpha)*X_bin_solar;

# Append data
X_valid_solar_norm = [X_num_solar X_bin_solar];

In [75]:
#scale Y 
y_train_solar_norm = min_max_scaling(Matrix(y_train_solar), 1:1)
y_valid_solar_norm = min_max_scaling(Matrix(y_valid_solar), 1:1);

#### WIND

In [76]:
X_num_wind=min_max_scaling(Matrix(X_train_wind), 1:15)

# Binary features as they are
X_bin_wind = X_train_wind[:,16:end]

# Numerical vs Categorical features scaling
alpha = 0.75
X_num_wind = alpha*X_num_wind
X_bin_wind = (1-alpha)*X_bin_wind;

# Append data
X_train_wind_norm = [X_num_wind X_bin_wind];

In [77]:
X_num_wind=min_max_scaling(Matrix(X_valid_wind), 1:15)

# Binary features as they are
X_bin_wind = X_valid_wind[:,16:end]

# Numerical vs Categorical features scaling
alpha = 0.75
X_num_wind = alpha*X_num_wind
X_bin_wind = (1-alpha)*X_bin_wind;

# Append data
X_valid_wind_norm = [X_num_wind X_bin_wind];

In [78]:
#scale Y 
y_train_wind_norm=min_max_scaling(Matrix(y_train_wind), 1:1)
y_valid_wind_norm=min_max_scaling(Matrix(y_valid_wind), 1:1);

## Holistic

In [79]:
function correlation(X,p)
    correlation_matrix = Statistics.cor(X) #compute corr matrix
    n,m = size(correlation_matrix) #set sizes
    correlated_pairs=[] #empty list to store correlated pairs
    for i=1:n #for all features
        for j=i+1:m 
            if abs(correlation_matrix[i,j])>p #if the abs value of the corr is higher than p
                push!(correlated_pairs, (i,j)) #append pair to list of correlated pairs
            end
        end
    end
    return correlated_pairs
end

correlation (generic function with 1 method)

In [80]:
#Build transformation function
function transformation(X)
    X_old=DataFrame(X, :auto) #define X as a df
    X_new=DataFrame() #new empty df 
    n,p=size(X_old)
    e=1
    for i=1:p #for each feature in X add 4 transformations
        X_new[!, "X$i"]=X_old[:,i] #transformation 1: no transformation
        X_new[!, "Sqrt$i"]=X_old[:,i].^2 #transformation 2: square root
        #X_new[!, "Log$i"]=log.(abs.(X_old[:,i])) #transformation 3: log
        X_new[!, "Abs_Sqrt$i"]=sqrt.(abs.(X_old[:,i])) #transformation 3: absolute squared root
        #square root of absolute value
        
    end
    return(X_new) #we return a new df with all transformations, it will have size nxp*4
end

transformation (generic function with 1 method)

In [81]:
function holistic_regression(X,y,lambda=0.25, per=0.6, M=50, k=10) #add parameter per
    
    #Call functions
    X_new=Matrix(transformation(X)) #call function with all transformations of X

    HC = correlation(X_new, per) #call correlation function to compute hc_pairs

    
    #Set sizes
    n,p_new=size(X_new)
    
    #Build model
    model = Model(Gurobi.Optimizer)
   
    #model = Model(Gurobi.Optimizer, NonConvex = 2)#we have defined the model, pass Gurobi optimizer into the model
    #model = Model(with_optimizer(Gurobi.Optimizer, NonConvex = 2))
    set_optimizer_attribute(model,"OutputFlag",0)
    
    #Insert variables
    @variable(model, beta[1:p_new])
    @variable(model, beta_abs[1:p_new])
    @variable(model, z[1:p_new], Bin) #we add a binary variable

    

    #Insert constraints
    @constraint(model, beta_abs .>= beta) #put the dot is like doing the loop over all j
    @constraint(model, beta_abs .>= -beta)
    
    
        
    #sparsity constraint: over all 60 features (including transformations) 
    @constraint(model, -M*z .<= beta)
    @constraint(model, beta .<= M*z)
    @constraint(model, sum(z) <= k) 
    
    
    #constraint on Transformation: from the 4 transformations per each feature we only select one
    for i=1:3:p_new
        #for j=i:i+3 #for every 4 transformations
        @constraint(model, sum(z[i:i+2])<=1)
        #end
    end #we get a vector with 15 features
    
    #constraint on HC pairs once we have selected 15 features 
    for (i,j) in HC
        @constraint(model, z[i] + z[j] <= 1)
    end #we can only take one of the pairs of correlated pairs
    #@constraint(model, sum(z[i])<=k) #ensure that the model has at most 8 features
    
        
    #Insert objective
    @objective(model, Min, sum((y-X_new*beta).^2) + lambda*sum(beta_abs))
    
    
    
    # Optimize
    optimize!(model)
    println("runs5")
    
    # Return estimated betas
    return (value.(beta))
    
end

holistic_regression (generic function with 5 methods)

In [17]:
#find best lambda, rho, and k
function holistic_cv(X,y,X_valid,y_valid)
    lowest_mse=Inf
    best_lambda=0
    best_rho=0
    best_k=0
    for lambda in [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
        for rho in [0.5, 0.6, 0.7, 0.8]
            for k in [3,4,5,6,7,8]
                beta_opt=holistic_regression(X, y, lambda, rho, 50, k)
                X_valid_trans=Matrix(transformation(X_valid))
                mse_temp= mse(y_valid, X_valid_trans*beta_opt)
                #for each lambda, find the best rho
                if mse_temp<lowest_mse
                    lowest_mse=mse_temp
                    best_lambda=lambda
                    best_rho=rho
                    best_k=k
                end
            end
        end
    end
    return best_lambda, best_rho, best_k
end


holistic_cv (generic function with 1 method)

### SOLAR

#### CV

In [13]:
#Not normalized
holistic_cv(X_train_solar,y_train_solar, X_valid_solar, y_valid_solar)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18

(0.1, 0.5, 3)

In [14]:
#X normalized
holistic_cv(X_train_solar_norm,y_train_solar, X_valid_solar_norm, y_valid_solar)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18

(0.1, 0.5, 8)

#### EVALUATION 

In [82]:
#Not normalized
beta=holistic_regression(X_train_solar,y_train_solar, 0.1, 0.5, 30, 3);

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
runs5


In [83]:
println("MSE: ", mse(y_valid_solar, Matrix(transformation(X_valid_solar))*beta))
println("MAE: ", mae(y_valid_solar, Matrix(transformation(X_valid_solar))*beta))
println("R2: ", r2_score(y_valid_solar, Matrix(transformation(X_valid_solar))*beta))

MSE: 9.007675893550228e6
MAE: 2997.9146689497716
R2: -1.421466481550252e26


In [86]:
#X normalized
beta=holistic_regression(X_train_solar_norm,y_train_solar, 0.1, 0.5, 50, 9);

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
runs5


In [87]:
println("MSE: ", mse(y_valid_solar, Matrix(transformation(X_valid_solar_norm))*beta))
println("MAE: ", mae(y_valid_solar, Matrix(transformation(X_valid_solar_norm))*beta))
println("R2: ", r2_score(y_valid_solar, Matrix(transformation(X_valid_solar_norm))*beta))

MSE: 7.340355548872982e6
MAE: 2705.7272885964035
R2: -6004.372408067889


In [20]:
selected_features=findall(x->x!=0, beta)
print(beta[selected_features])
names(transformation(X_train_solar))[selected_features]

[50.0, 50.0, 50.0, 50.0]

4-element Vector{String}:
 "Abs_Sqrt4"
 "Abs_Sqrt12"
 "Abs_Sqrt13"
 "Abs_Sqrt23"

In [21]:
#X,y normalized
beta=holistic_regression(X_train_solar_norm,y_train_solar_norm, 0.1, 0.5, 50, 8);

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18


In [22]:
println("MSE: ", mse(y_valid_solar_norm, Matrix(transformation(X_valid_solar_norm))*beta))
println("MAE: ", mae(y_valid_solar_norm, Matrix(transformation(X_valid_solar_norm))*beta))
println("R2: ", r2_score(y_valid_solar_norm, Matrix(transformation(X_valid_solar_norm))*beta))

MSE: 0.050112259588792854
MAE: 0.18115855434991734
R2: -38.07066888786043


### WIND

#### CV

In [88]:
#Not normalized
holistic_cv(X_train_wind,y_train_wind, X_valid_wind, y_valid_wind)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
runs5
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
runs5
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
runs5
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
runs5
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
runs5
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
runs5
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
runs5
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
runs5
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
runs5
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
runs5
Set parameter Username
Academi

(0.1, 0.5, 3)

In [89]:
#X normalized
holistic_cv(X_train_wind_norm,y_train_wind, X_valid_wind_norm, y_valid_wind)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
runs5
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
runs5
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
runs5
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
runs5
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
runs5
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
runs5
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
runs5
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
runs5
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
runs5
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
runs5
Set parameter Username
Academi

(0.1, 0.5, 8)

In [25]:
#Both Normalized
holistic_cv(X_train_wind_norm,y_train_wind_norm, X_valid_wind_norm, y_valid_wind_norm)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18

(0.9, 0.5, 6)

#### EVALUATION 

In [96]:
#Not normalized
beta=holistic_regression(X_train_wind,y_train_wind, 0.1, 0.5,50, 3);

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
runs5


In [27]:
selected_features=findall(x->x!=0, beta)
names(transformation(X_train_solar))[selected_features]

String[]

In [97]:
println("MSE: ", mse(y_valid_wind, Matrix(transformation(X_valid_wind))*beta))
println("MAE: ", mae(y_valid_wind, Matrix(transformation(X_valid_wind))*beta))
println("R2: ", r2_score(y_valid_wind, Matrix(transformation(X_valid_wind))*beta))

MSE: 8.980217836757991e6
MAE: 2981.6388888888887
R2: -1.0628500738834906e26


In [102]:
#X normalized
beta=holistic_regression(X_train_wind_norm,y_train_wind, 0.1, 0.5,50, 9);

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
runs5


In [103]:
println("MSE: ", mse(y_valid_wind, Matrix(transformation(X_valid_wind_norm))*beta))
println("MAE: ", mae(y_valid_wind, Matrix(transformation(X_valid_wind_norm))*beta))
println("R2: ", r2_score(y_valid_wind, Matrix(transformation(X_valid_wind_norm))*beta))

MSE: 7.355745286395899e6
MAE: 2695.904677713879
R2: -6198.440895994354


In [31]:
selected=findall(x->x!=0, beta)
print(beta[selected])
names(transformation(X_train_wind))[selected]

[50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0]

9-element Vector{String}:
 "Abs_Sqrt3"
 "Abs_Sqrt4"
 "Abs_Sqrt9"
 "Abs_Sqrt10"
 "Abs_Sqrt11"
 "Abs_Sqrt12"
 "Abs_Sqrt13"
 "Abs_Sqrt18"
 "Abs_Sqrt23"

In [32]:
#X,y normalized
beta=holistic_regression(X_train_wind_norm,y_train_wind_norm, 0.9, 0.9,50, 7);

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18


In [33]:
selected=findall(x->x!=0, beta)
println(beta[selected])
names(transformation(X_train_wind))[selected]

[-0.14126080716510214, 2.2266845448595576, 0.14209278603792305, -0.17177960457664865, 0.17200287641833154, -0.2072149348156093, -0.0386519375165307]


7-element Vector{String}:
 "Sqrt3"
 "X9"
 "Abs_Sqrt10"
 "X11"
 "Sqrt13"
 "Abs_Sqrt14"
 "Abs_Sqrt17"

In [34]:
beta

111-element Vector{Float64}:
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
 -0.14126080716510214
  0.0
  0.0
  0.0
  0.0
  0.0
  ⋮
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0

In [35]:
println("MSE: ", mse(y_valid_wind_norm, Matrix(transformation(X_valid_wind_norm))*beta))
println("MAE: ", mae(y_valid_wind_norm, Matrix(transformation(X_valid_wind_norm))*beta))
println("R2: ", r2_score(y_valid_wind_norm, Matrix(transformation(X_valid_wind_norm))*beta))

MSE: 0.09947381151364786
MAE: 0.23244004598036627
R2: -0.28715991383312556


## TEST VALIDATION

In [3]:
X_total_solar=Matrix(DataFrame(CSV.File("X_total_solar.csv")))
y_total_solar=Matrix(DataFrame(CSV.File("y_total_solar.csv")))
X_total_wind=Matrix(DataFrame(CSV.File("X_total_wind.csv")))
y_total_wind=Matrix(DataFrame(CSV.File("y_total_wind.csv")))

X_test_solar=Matrix(DataFrame(CSV.File("X_test_solar.csv")))
y_test_solar=Matrix(DataFrame(CSV.File("y_test_solar.csv")))
X_test_wind=Matrix(DataFrame(CSV.File("X_test_wind.csv")))
y_test_wind=Matrix(DataFrame(CSV.File("y_test_wind.csv")));

#### SOLAR

#### Not Normalized

In [104]:
beta=holistic_regression(X_train_solar,y_train_solar, 0.1, 0.9, 50, 3);

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
runs5


In [54]:
mse(y_test_solar, Matrix(transformation(X_test_solar))*beta)

1.1320372607876712e7

In [55]:
mae(y_test_solar, Matrix(transformation(X_test_solar))*beta)

3332.23401826484

In [ ]:
#Not normalized
beta=holistic_regression(X_total_solar,y_total_solar, 0.1, 0.9, 50, 8);

In [105]:
println("MSE: ", mse(y_test_solar, Matrix(transformation(X_test_solar))*beta))
println("MAE: ", mae(y_test_solar, Matrix(transformation(X_test_solar))*beta))
println("R2: ", r2_score(y_test_solar, Matrix(transformation(X_test_solar))*beta))

MSE: 1.1320372607876712e7
MAE: 3332.23401826484
R2: -1.7864242020828468e26


#### Normalized

In [112]:
#X normalized
beta=holistic_regression(X_train_solar_norm,y_train_solar, 0.1, 0.9, 50, 3);

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
runs5


In [109]:
#scale SOLAR TEST only numerical
X_test_num_solar=min_max_scaling(Matrix(X_test_solar), 1:15)

# Binary features as they are
X_test_bin_solar = X_test_solar[:,16:end]

# Numerical vs Categorical features scaling
alpha = 0.75
X_test_num_solar = alpha*X_test_num_solar
X_test_bin_solar = (1-alpha)*X_test_bin_solar;

# Append data
X_test_solar_norm = [X_test_num_solar X_test_bin_solar];

In [113]:
println("MSE: ", mse(y_test_solar, Matrix(transformation(X_test_solar_norm))*beta))
println("MAE: ", mae(y_test_solar, Matrix(transformation(X_test_solar_norm))*beta))
println("R2: ", r2_score(y_test_solar, Matrix(transformation(X_test_solar_norm))*beta))

MSE: 1.0565209398804456e7
MAE: 3217.0214604307193
R2: -39070.728047839795


In [43]:
#X,y normalized
beta=holistic_regression(X_total_wind[1:3000,:],y_total_wind[1:3000], 0.1, 0.5, 50, 8);

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
runs5


## WIND

#### Not Normalized

In [120]:
beta = holistic_regression(X_train_wind,y_train_wind, 0.1, 0.5, 50, 3);

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
runs5


In [119]:
println("MSE: ", mse(y_test_wind, Matrix(transformation(X_test_wind))*beta))
println("MAE: ", mae(y_test_wind, Matrix(transformation(X_test_wind))*beta))
println("R2: ", r2_score(y_test_wind, Matrix(transformation(X_test_wind))*beta))

MSE: 6.2935853668188736e7
MAE: 5958.99695585997
R2: -7.448747673731694e26


In [121]:
beta = holistic_regression(X_train_wind_norm,y_train_wind, 0.1, 0.5, 50, 9);

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
runs5


In [122]:
#scale WIND TEST only numerical
X_test_num_wind=min_max_scaling(Matrix(X_test_wind), 1:15)

# Binary features as they are
X_test_bin_wind = X_test_wind[:,16:end]

# Numerical vs Categorical features scaling
alpha = 0.75
X_test_num_wind = alpha*X_test_num_wind
X_test_bin_wind = (1-alpha)*X_test_bin_wind;

# Append data
X_test_wind_norm = [X_test_num_wind X_test_bin_wind];

In [123]:
println("MSE: ", mse(y_test_wind, Matrix(transformation(X_test_wind_norm))*beta))
println("MAE: ", mae(y_test_wind, Matrix(transformation(X_test_wind_norm))*beta))
println("R2: ", r2_score(y_test_wind, Matrix(transformation(X_test_wind_norm))*beta))

MSE: 5.947941917450608e7
MAE: 5674.5866695556415
R2: -55091.63199087466
